In [1]:
%pip install sentence-transformers
%pip install llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.0.1
    Uninstalling pypdf-5.0.1:
      Successfully uninstalled pypdf-5.0.1
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
 

In [2]:
import nltk
nltk.download('punkt_tab')
import os
os.environ["WANDB_MODE"] = "disabled"
import torch
from sentence_transformers import SentenceTransformer, models, datasets, losses
from torch.utils.data import DataLoader
import pickle
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
csv_path = "/kaggle/input/qa-testset/Hydrogen_LLM.csv"
data = pd.read_csv(csv_path)

def calculate_mrr(question_embeddings, answer_embeddings):
    reciprocal_ranks = []

    for i, question_embedding in enumerate(question_embeddings):
        similarities = cosine_similarity([question_embedding], answer_embeddings)[0]
        sorted_indices = np.argsort(-similarities)
        rank_of_correct_answer = np.where(sorted_indices == i)[0][0] + 1 
        reciprocal_ranks.append(1 / rank_of_correct_answer)
        
    mrr = np.mean(reciprocal_ranks)
    return mrr


def calculate_recall_at_k(question_embeddings, answer_embeddings, K):
    recall_scores = []
    for i, question_embedding in enumerate(question_embeddings):
        similarities = cosine_similarity([question_embedding], answer_embeddings)[0]
        sorted_indices = np.argsort(-similarities)
        if i in sorted_indices[:K]:
            recall_scores.append(1)  # Câu trả lời đúng nằm trong top K
        else:
            recall_scores.append(0)  # Câu trả lời đúng không nằm trong top K
    
    recall_at_k = np.mean(recall_scores)
    return recall_at_k


def test_model (model):
    questions_embeddings = model.encode(data['Questions'].tolist(), convert_to_tensor=True)

    answers_embeddings = model.encode(data['Answer'].tolist(), convert_to_tensor=True)

    questions_embeddings = questions_embeddings.cpu().numpy()
    answers_embeddings = answers_embeddings.cpu().numpy()

    mrr = calculate_mrr(questions_embeddings, answers_embeddings)
    recall_1 = calculate_recall_at_k(questions_embeddings, answers_embeddings, 1)
    recall_3 = calculate_recall_at_k(questions_embeddings, answers_embeddings, 3)
    recall_5 = calculate_recall_at_k(questions_embeddings, answers_embeddings, 5)

    print (f"MRR: {mrr}")
    print(f"Recall@5: {recall_5}")
    print(f"Recall@3: {recall_3}")
    print(f"Recall@1: {recall_1}")

In [4]:
def main():
    # Set up argparse to receive command-line arguments
    input_file = "/kaggle/input/rag-hydrogen/chunkers_final.pkl"
    output_dir = "/kaggle/working/output_model"

    epochs = 15
    learning_rate = 3e-5
    batch_size = 2
    # Check for GPU and use it
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Training on {device}...")

    # Read input file containing training sentences
    with open(input_file, 'rb') as file:
        documents = pickle.load(file)
        train_sentences = []
        for doc in documents:
            train_sentences.append(doc.text)

    # Define TSDAE model with "CLS" pooling and use fp16
    model_name = "BAAI/bge-base-en"
    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "cls")
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    # Test base model
    test_model(model)
    
    # Create dataset with DenoisingAutoEncoder
    train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
    # DataLoader to split data into batches
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Loss function for TSDAE
    train_loss = losses.DenoisingAutoEncoderLoss(
        model, decoder_name_or_path=model_name, tie_encoder_decoder=True
    )
    

    for epoch in range(epochs):
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=1,
            weight_decay=0,
            scheduler="warmupcosine",
            optimizer_params={"lr": learning_rate},
            show_progress_bar=True,
            use_amp=True,
            checkpoint_path = "/kaggle/working/checkpoints",
        )
        # Test the model with Q&A testset
        test_model(model)
        # Save the model after each epoch
        model.save(f"{output_dir}/epoch_{epoch + 1}")
        print(f"Model after epoch {epoch + 1} saved to {output_dir}/epoch_{epoch + 1}")
        torch.cuda.empty_cache()
        

if __name__ == "__main__":
    main()

Training on cuda...


config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
500,4.159700
1000,4.049100
1500,3.919500
2000,3.848500
2500,3.771100


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

MRR: 0.42652682164097044
Recall@5: 0.6283185840707964
Recall@3: 0.504424778761062
Recall@1: 0.2743362831858407
Model after epoch 15 saved to /kaggle/working/output_model/epoch_15
